In [ ]:
!apt-get update && apt-get install pciutils coreutils -y
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve &

In [ ]:
!echo llama3:8b > llm

In [ ]:
!echo dolphin-phi > llm

In [ ]:
!nohup ollama serve &

In [ ]:
!ollama list && ollama pull $(cat llm) 2>/dev/null && ollama list

In [ ]:
!bash -c 'lsof -i -n  | grep ollama'

In [ ]:
# Example of curl to local to test the model. Make sure to use the correct model. Change to ngrok URL if needed.
!curl -X POST http://localhost:11434/api/generate -d '{ "model": "llama3:8b", "prompt": "Why the sky is blue? make a very short awswer with 50 tokens", "stream": false }'

In [ ]:
!pip install -q aiohttp pyngrok

In [ ]:
import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

#register an account at ngrok.com and create an authtoken and place it here
await asyncio.gather(
    run_process(['ngrok', 'config', 'add-authtoken','REPLACE_YOUR_NGROK_TOKEN_HERE'])
)

await asyncio.gather(
    #run_process(['ollama', 'serve']),
    run_process(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header', 'localhost:11434'])
)